In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('test').getOrCreate()

In [3]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
spark.sparkContext._conf.getAll()

[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.app.name', 'test'),
 ('spark.driver.host', 'PF28CEFN.mshome.net'),
 ('spark.executor.id', 'driver'),
 ('spark.app.id', 'local-1669980639542'),
 ('spark.app.submitTime', '1669980637109'),
 ('spark.driver.port', '50369'),
 ('s

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as F
from logging import Logger
import os

In [3]:
from pyspark.sql import DataFrame

In [1]:
source_config = {
        "url" : "jdbc:postgresql://localhost:5432/dbt-workshop",
        "driver" : "org.postgresql.Driver",
        "user" : "postgres",
        "password" : "admin"
    }

target_config = {
        "url" : "jdbc:postgresql://localhost:5432/dbt-workshop",
        "driver" : "org.postgresql.Driver",
        "user" : "postgres",
        "password" : "admin"
    }

In [4]:
def read_from_pg(config: dict, sql: str, table: str,spark) -> DataFrame:
    """ Read dataframe from postgres
    Args:
        config: settings for connect
        sql: sql to read, it may be one of these format
             - 'table_name'
             - 'schema_name.table_name'
             - '(select a, b, c from t1 join t2 ...) as foo'
        spark: specific current spark_context or None
        logger: logger
    Returns:
        selected DF
    """
    sql = ''
    # if logger:
    #     logger.info('read_from_pg:\n{}'.format(sql))
        
    if sql:
        source_df = spark.read.format("jdbc").options(**config).option("query", sql).load()
        
    else:
        source_df = spark.read.format("jdbc").options(**config).option('dbtable',table).load()
        
    return source_df 



def write_to_pg(df: DataFrame, config: dict, table: str, column_list: list, mode: str='append' ) -> None:
    """ Write dataframe to postgres
    Args:
        df: DataFrame to write
        config: config dict
        table: table_name in which we write_data
        column_list: list of columns in which we write the data
        logger: logger
        mode: mode, one of these:
            - append - create table if not exists (with all columns of DataFrame)
                and write records to table (using fields only in table columns)
            - overwrite - truncate table (if exists) and write records (using fields only in table columns)
            - overwrite_full - drop table and create new one with all columns and DataFrame and append records to it
            - fail - fail if table is not exists, otherwise append records to it
    """
    try:
        # df.select(*column_list).write.format('jdbc').options(**config).option('dbtable',table).mode(mode).save()

        df.select(*column_list).write.format('jdbc').options(
                                        url = "jdbc:postgresql://localhost:5432/dbt-workshop",
                                        driver = "org.postgresql.Driver",
                                        user = "postgres",
                                        password = "admin"
                                        ).option('dbtable',table).mode(mode).save()

        return "data written into postgresql successfully"
    except Exception as e:
        return f"{e}"


In [5]:
SPARK_CONFIG = {
    "MASTER": "local[*]",
    "settings": {
      "spark.executor.cores": "1",
      "spark.executor.memory": "1g",
      "spark.driver.cores": "1",
      "spark.driver.memory": "1g",
      "spark.cores.max": "1"
    }
}


def init_spark_context(appname: str, jar_path:str) -> SparkContext:
    """ init spark context """

    os.environ['PYSPARK_SUBMIT_ARGS'] = f'--jars {jar_path} pyspark-shell'
    conf = SparkConf()
    conf.setMaster(SPARK_CONFIG['MASTER'])
    conf.setAppName(appname)

    for setting, value in SPARK_CONFIG['settings'].items():
        conf.set(setting, value)

    sc = SparkContext(conf=conf)

    return sc

jar_path = 'jars/postgresql-42.5.0.jar'
sc = init_spark_context('app', jar_path)
spark = SparkSession(sc)

In [7]:
table = "dev.stripe_payments"
df = read_from_pg(source_config, '' , table, spark)
print(df.printSchema())

root
 |-- id: integer (nullable = true)
 |-- orderid: integer (nullable = true)
 |-- paymentmethod: string (nullable = true)
 |-- status: string (nullable = true)
 |-- amount: integer (nullable = true)
 |-- created: date (nullable = true)

None


In [9]:
columns_list = df.columns

In [13]:
write_to_pg(df, target_config, 'dev.test_table', columns_list, 'append')

'data written into postgresql successfully'

In [5]:
from logging import Logger
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame

In [9]:
def write_to_pg(spark:SparkSession, df: DataFrame, config: dict, table: str, mode: str='append' ) -> None:
    """ Write dataframe to postgres
    Args:
        df: DataFrame to write
        config: config dict
        table: table_name in which we write_data
        column_list: list of columns in which we write the data
        mode: mode, one of these:
            - append - create table if not exists (with all columns of DataFrame)
                and write records to table (using fields only in table columns)
            - overwrite - truncate table (if exists) and write records (using fields only in table columns)
            - overwrite_full - drop table and create new one with all columns and DataFrame and append records to it
            - fail - fail if table is not exists, otherwise append records to it
    """
    
    try:
        column_list = df.columns
        if len(column_list) == 0:
            return("No columns to write into")

        else:
            df.select(*column_list).write.format('jdbc').options(**config).option('dbtable',table).mode(mode).save()
            return "Data written into postgresql successfully"
        
    except Exception as e:
        print("Failure occured check logs")
        return f"{e}"

In [16]:
config = {
            "url": "jdbc:postgresql://localhost:5432/postgres",
            "driver": "org.postgresql.Driver",
            "user": "postgres",
            "password": "admin"
}
table = 'emp_details'
mode = 'append'
sql = ''

In [21]:
df = spark.read.option('header',True).csv('source_data.csv', inferSchema =True)

In [22]:
df.printSchema()

root
 |-- fname: string (nullable = true)
 |-- lname: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- dob: string (nullable = true)
 |-- salary: double (nullable = true)



In [23]:
# write_to_pg(spark, df, config, table, mode)

In [86]:
def read_from_pg(spark:SparkSession, config: dict, sql: str, table: str) -> DataFrame:
    """ Read dataframe from postgres
    Args:
        config: settings for connect
        sql: sql to read, it may be one of these format
             - 'table_name'
             - 'schema_name.table_name'
             - '(select a, b, c from t1 join t2 ...) as foo'
        spark: specific current spark_context or None
    Returns:
        selected DF
    """        
    print("reading from postgresql")
    if sql:
        print("executing query to create df")
        source_df = spark.read.format("jdbc").options(**config).option("query", sql).load()
        
    else:
        print("reading directly from source table")
        source_df = spark.read.format("jdbc").options(**config).option('dbtable',table).load()
        
    return source_df 


In [63]:
source_config = {
            "url": "jdbc:postgresql://localhost:5432/postgres",
            "driver": "org.postgresql.Driver",
            "user": "postgres",
            "password": "admin"
}
source_table = 'source_emp_details'
sql = ''

In [36]:
source_df = read_from_pg(spark, source_config, sql, source_table)

reading from postgresql
reading directly from source table


In [37]:
source_df.schema

StructType([StructField('fname', StringType(), True), StructField('lname', StringType(), True), StructField('age', StringType(), True), StructField('dob', StringType(), True), StructField('salary', StringType(), True)])

In [84]:
source_df.printSchema()

root
 |-- fname: string (nullable = true)
 |-- lname: string (nullable = true)
 |-- age: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- salary: string (nullable = true)



In [69]:
source_df.show()

+------+---------+---+----------+-------+
| fname|    lname|age|       dob| salary|
+------+---------+---+----------+-------+
|  vish|dabholkar| 25|21-02-1998|   1000|
|akshay|dabholkar| 28|25-03-1995|2500.56|
|   sam|    patil| 24|27-09-1999| 524.44|
+------+---------+---+----------+-------+



In [72]:
target_config = {
            "url": "jdbc:postgresql://localhost:5432/postgres",
            "driver": "org.postgresql.Driver",
            "user": "postgres",
            "password": "admin"
}
target_table = 'target_emp_details'
sql = ''

In [41]:
target_df = read_from_pg(spark, target_config, sql, target_table)

reading from postgresql
reading directly from source table


In [76]:
target_df.printSchema()

root
 |-- fname: string (nullable = true)
 |-- lname: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- dob: date (nullable = true)
 |-- salary: double (nullable = true)



In [51]:
target_df.schema

pyspark.sql.types.StructType

In [88]:
def get_mappings(spark:SparkSession):
    mapping_file_path = 'mapping.csv'
    mapping_df = spark.read.option('header',True).csv(mapping_file_path)
    source_column_list = list(mapping_df.select('source').toPandas()['source'])
    target_column_list = list(mapping_df.select('target').toPandas()['target'])
    source_target_dict = dict(zip(source_column_list, target_column_list))
    return source_target_dict

mapping = get_mappings(spark)

In [101]:
source_df = read_from_pg(spark, source_config, sql, source_table)

reading from postgresql
reading directly from source table


In [102]:
from pyspark.sql.functions import col

In [108]:
target_df.select([col(c).cast("string") for c in target_df.columns])

DataFrame[fname: string, lname: string, age: string, dob: string, salary: string]

In [109]:
df.withColumn("id", F.col("new_id").cast(T.StringType()))

root
 |-- fname: string (nullable = true)
 |-- lname: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- dob: date (nullable = true)
 |-- salary: double (nullable = true)



In [112]:
target_df.dtypes

[('fname', 'string'),
 ('lname', 'string'),
 ('age', 'int'),
 ('dob', 'date'),
 ('salary', 'double')]

In [113]:
source_df = source_df.withColumn("salary",source_df.salary.cast('double'))

In [114]:
source_df.printSchema()

root
 |-- fname: string (nullable = true)
 |-- lname: string (nullable = true)
 |-- age: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- salary: double (nullable = true)



In [ ]:
import cx_Oracle
import psycopg2

In [44]:
def get_pg_table_metadeta(config:dict, table_name):
    """
    Function to get table metadeta from a postgresql table
    Args:
        config: It is a dict with parameters required to connect to a postgres sever
            like hostname, db name, username and password
            
        table_name: name of the table for which we want to get the metadata
        
    return: 
        pandas dataframe for postgresql table metadeta
    """
    try:
        
        host = config['host']
        database = config['database']
        user = config['user']
        password = config['password']

        conn = psycopg2.connect(
                    host=host,
                    database=database,
                    user=user,
                    password=password)
        cur = conn.cursor()

        query = f''' select column_name, data_type, character_maximum_length, column_default, is_nullable
    from INFORMATION_SCHEMA.COLUMNS where table_name = '{table_name}'; '''
        df = pd.read_sql_query(query ,con=conn)
        conn.close()
        return df
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')

In [63]:
def get_oracle_table_metadeta(config:dict, table_name):
    """
    Function to get table metadeta from a oracle db table
    Args:
        config: It is a dict with parameters required to connect to a oracle db sever
            like dsn, username and password
            
        table_name: name of the table for which we want to get the metadata
        
    return: 
        pandas dataframe for postgresql table metadeta
    """
    try:
        dsn = config['dsn']
        username = config['user']
        password = config['password']
        
        conn = cx_Oracle.connect(user=username, 
                                 password=password,
                               dsn=dsn,
                               encoding="UTF-8")
        cur = conn.cursor()

        query = f'''  select 
            t.table_name as table_name  
              , t.column_name                        
              , t.data_type
              , cc.constraint_name
              , uc.constraint_type
           from user_tab_columns t
                left join user_cons_columns cc
                  on (cc.table_name = t.table_name and
                      cc.column_name = t.column_name)
                left join user_constraints uc
                  on (t.table_name = uc.table_name and
                      uc.constraint_name = cc.constraint_name )
         where t.table_name in ('{table_name}')'''

        df = pd.read_sql_query(query ,con=conn)
        return df
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')

In [ ]:
def get_sql_sever_table_metadeta(config:dict, table_name):
    """
    Function to get table metadeta from a sql_sever db table
    Args:
        config: It is a dict with parameters required to connect to a sql_sever 
            like host, database, username and password
            
        table_name: name of the table for which we want to get the metadata
        
    return: 
        pandas dataframe for postgresql table metadeta
    """
    try:
        conn = pyodbc.connect(
                "Driver={ODBC Driver 18 for SQL Server};"
                        f"Server={host};"
                        f"Database={database};"
                        f"uid={user};pwd={password}")
        cur = conn.cursor()
        
        query = f'''  '{table_name}' '''

        df = pd.read_sql_query(query ,con=conn)
        return df
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')

In [ ]:
f"select * from INFORMATION_SCHEMA.COLUMNS where table_name = {table_name}"


In [52]:
username = 'cmx_ors'
password = 'cmx_ors'
dsn = '192.168.2.70:1521/PDB'
table_name = 'C_BO_ADDR'

In [60]:
try:
    conn = cx_Oracle.connect(user=username, 
                             password=password,
                           dsn=dsn,
                           encoding="UTF-8")
    cur = conn.cursor()

    query = f'''  select 
    t.table_name as table_name  
      , t.column_name                        
      , t.data_type
      , cc.constraint_name
      , uc.constraint_type
   from user_tab_columns t
        left join user_cons_columns cc
          on (cc.table_name = t.table_name and
              cc.column_name = t.column_name)
        left join user_constraints uc
          on (t.table_name = uc.table_name and
              uc.constraint_name = cc.constraint_name )
 where t.table_name in ('{table_name}')'''
    print(query)
    df = pd.read_sql_query(query ,con=conn)
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()
        print('Database connection closed.')

  select 
    t.table_name as table_name  
      , t.column_name                        
      , t.data_type
      , cc.constraint_name
      , uc.constraint_type
   from user_tab_columns t
        left join user_cons_columns cc
          on (cc.table_name = t.table_name and
              cc.column_name = t.column_name)
        left join user_constraints uc
          on (t.table_name = uc.table_name and
              uc.constraint_name = cc.constraint_name )
 where t.table_name in ('C_BO_ADDR')


C:\Users\vishwajeet.dabholkar\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Database connection closed.


In [62]:
df.count()

TABLE_NAME         23
COLUMN_NAME        23
DATA_TYPE          23
CONSTRAINT_NAME     5
CONSTRAINT_TYPE     5
dtype: int64

In [30]:
host = 'localhost'
database = 'postgres'
user = 'postgres'
password = 'admin'
table_name = 'target_emp_details'


conn = psycopg2.connect(
                host=host,
                database=database,
                user=user,
                password=password)
cur = conn.cursor()


query = f''' select column_name, data_type, character_maximum_length, column_default, is_nullable
from INFORMATION_SCHEMA.COLUMNS where table_name = '{table_name}'; '''
query


cur.execute(query)

In [29]:
conn.close()

In [28]:
import pandas as pd

In [39]:
df = pd.read_sql_query(query ,con=conn)

C:\Users\vishwajeet.dabholkar\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [40]:
df.head()

,column_name,data_type,character_maximum_length,column_default,is_nullable
0,age,integer,NaN,None,YES
1,dob,date,NaN,None,YES
2,salary,double precision,NaN,None,YES
3,fname,character varying,50.0,None,YES
4,lname,character varying,50.0,None,YES


In [42]:
df.style.hide_index()

C:\Users\vishwajeet.dabholkar\AppData\Local\Temp\ipykernel_12948\496160328.py:1: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  df.style.hide_index()


column_name,data_type,character_maximum_length,column_default,is_nullable
age,integer,nan,None,YES
dob,date,nan,None,YES
salary,double precision,nan,None,YES
fname,character varying,50.000000,None,YES
lname,character varying,50.000000,None,YES


In [5]:
actual= ['CREATE_DATE','DELETED_DATE','CITY','CNTRY','STATE','PIN_CD','DIRTY_IND', 'dob']
predicted = ['created_date','deleted_date','city','country','state','pincode','dirty_ind','source_name', 'date_of_birth']

In [6]:
from fuzzywuzzy import fuzz
import csv

threshold = 80
result = {}
for act in actual:
    for pre in predicted:
        # print(act,':',pre)
        if pre in actual:
            result[act] : pre
        
        elif fuzz.token_set_ratio(act.lower(),pre.lower()) >= threshold:            
            result[act] = pre
            
        else:
            result[''] = pre
            
            
            
source = list(result.keys())
target = list(result.values())

with open(r'mapping_tg.csv', 'w', newline='') as csvfile:
    fieldnames = ['source','source_data_type','source_format','target','target_data_type','target_format','default_value']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for key, values in result.items():
        writer.writerow({'source':key, 'target':values})

In [7]:
result

{'CREATE_DATE': 'created_date',
 '': 'date_of_birth',
 'DELETED_DATE': 'deleted_date',
 'CITY': 'city',
 'CNTRY': 'country',
 'STATE': 'state',
 'DIRTY_IND': 'dirty_ind'}

In [93]:
from fuzzywuzzy import fuzz
import csv

def generate_mapping_file(actual, predicted):
    """
    Function to generate mapping file which will have source to target column mapping
    this mapping is atuo generated based on the column names.
    fucntion writes mapping into a file which should be validated by the user before using it for data migration
    Args:
        actual: source column name list
        predicted: target column list
    """
    try:        
        threshold = 80
        result = {}
        for act in actual:
            for pre in predicted:
                # print(act,':',pre)
                if pre in actual:
                    result[act] : pre

                elif fuzz.token_set_ratio(act.lower(),pre.lower()) >= threshold:            
                    result[act] = pre

                else:
                    result[''] = pre

        source = list(result.keys())
        target = list(result.values())

        with open(r'mapping_tg.csv', 'w', newline='') as csvfile:
            fieldnames = ['source','source_data_type','source_format','target','target_data_type','target_format','default_value']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for key, values in result.items():
                writer.writerow({'source':key, 'target':values})
                
    except Exception as e:
        print("Failure occured check logs")
        return f"{e}"